 Part A — Data Preparation
1. Objective

The goal of this section is to clean, align, and prepare both datasets for analysis. Since the trading dataset contains intraday trade-level data and the Fear & Greed dataset provides daily sentiment values, proper time alignment is required before performing performance and behavioral analysis.

2. Data Overview

Two datasets are provided:

Historical Trading Data: Contains individual trade records including account, trade size, side (long/short), timestamp, and closed PnL.

Fear & Greed Index Data: Contains daily market sentiment classification (Fear, Neutral, Greed).

The first step is to inspect dataset dimensions, check for missing values and duplicates, and understand overall data quality.

In [1]:
#Load datasets

import pandas as pd

trades = pd.read_csv("historical_data.csv", nrows=10000)
sentiment = pd.read_csv("fear_greed_index.csv")

In [2]:
trades.sample()

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
7659,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,85353.0,0.14301,12206.33,SELL,18-04-2025 00:10,89.71848,Close Long,53.285526,0x326186dc9ef500c5b12d0421bc3be20202bf00616ee0...,87215644427,True,2.807456,9.140000e+14,1.740000e+12


In [3]:
sentiment.sample()

,timestamp,value,classification,date
144,1530163800,20,Extreme Fear,2018-06-28


In [4]:
#Number of rows and columns
print("Trades dataset shape:", trades.shape)
print("Sentiment dataset shape:", sentiment.shape)

Trades dataset shape: (10000, 16)
Sentiment dataset shape: (2644, 4)


In [5]:
#missing values
trades.isna().sum()
sentiment.isna().sum()

,0
timestamp,0
value,0
classification,0
date,0


* No missing values were observed in either dataset, so no imputation or row removal was required.

In [6]:
#Duplicate Records
print("Trade duplicates:", trades.duplicated().sum())
print("Sentiment duplicates:", sentiment.duplicated().sum())


Trade duplicates: 0
Sentiment duplicates: 0


* Duplicate checks confirmed that both datasets contain unique records.

3. Timestamp Alignment Strategy

The trading dataset contains intraday timestamps, while the sentiment dataset is recorded at daily frequency. To ensure correct alignment without look-ahead bias, both timestamps will be converted to daily granularity. The datasets will then be merged on the date column using an inner join so that each trade is tagged with the sentiment of that day.



In [7]:
#Convert timestamps to daily date
trades["date"] = pd.to_datetime(trades["Timestamp"]).dt.date
sentiment["date"] = pd.to_datetime(sentiment["timestamp"]).dt.date


In [8]:
#Merge datasets on date
df = trades.merge(
    sentiment[["date", "classification"]],
    on="date",
    how="inner"
)

In [9]:
df.head(5)

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp,date,classification
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.0,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12,1970-01-01,Fear
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.0,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12,1970-01-01,Extreme Fear
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.0,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12,1970-01-01,Fear
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.0,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12,1970-01-01,Extreme Fear
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.0,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12,1970-01-01,Extreme Fear


* Trade-level data was aggregated to daily granularity and merged with the Fear & Greed Index using an inner join on date, ensuring that each trade is associated with the correct daily market sentiment.

4. Feature Engineering Plan

After merging the datasets, key performance and behavioral metrics will be created to support further analysis. These include:

* Daily PnL per trader

* Win rate

* Average trade size

* Trade frequency per day

* Long/short ratio

* Leverage distribution proxy

These metrics will allow us to compare trader performance and behavior across different sentiment regimes.

In [10]:
#Daily PnL per trader (account)
daily_pnl_trader = (
    df.groupby(["date", "Account"])["Closed PnL"]
    .sum()
    .reset_index(name="daily_pnl")
)

daily_pnl_trader

,date,Account,daily_pnl
0,1970-01-01,0x513b8629fe877bb581bf244e326a047b249c4ff1,1.448183e+09
1,1970-01-01,0x72c6a4624e1dffa724e6d00d64ceae698af892a0,1.065562e+09
2,1970-01-01,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,1.793838e+08


In [11]:
#Win rate
df["win"] = (df["Closed PnL"] > 0).astype(int)

win_rate = (
    df.groupby("Account")["win"]
    .mean()
    .reset_index(name="win_rate")
)

df["win"]

,win
0,0
1,0
2,0
3,0
4,0
...,...
26439995,1
26439996,1
26439997,1
26439998,1


In [12]:
#Average trade size

avg_trade_size = (
    df.groupby("Account")["Size USD"]
    .mean()
    .reset_index(name="avg_trade_size_usd")
)

avg_trade_size


,Account,avg_trade_size_usd
0,0x513b8629fe877bb581bf244e326a047b249c4ff1,32018.060852
1,0x72c6a4624e1dffa724e6d00d64ceae698af892a0,2133.667364
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,2979.441776


In [13]:
#Leverage distribution
leverage_dist = df["Size USD"].describe()
leverage_dist



,Size USD
count,2.644000e+07
mean,2.610972e+04
std,1.237189e+05
min,1.000000e-01
25%,4.965100e+02
50%,1.580400e+03
75%,9.999880e+03
max,3.921431e+06


In [14]:
#Number of trades per day

trades_per_day = (
    df.groupby("date")
    .size()
    .reset_index(name="trades_per_day")
)

trades_per_day

,date,trades_per_day
0,1970-01-01,26440000


In [15]:
#Long / Short ratio
long_short = (
    df.groupby(["date", "Side"])
    .size()
    .unstack(fill_value=0)
)

long_short["long_short_ratio"] = (
    long_short["BUY"] / long_short["SELL"]
)

PART B — Analysis

Compare performance metrics by sentiment

In [16]:
#Compare performance metrics by sentiment

# Aggregate performance by sentiment
performance_summary = df.groupby("classification").agg(
    avg_pnl=("Closed PnL", "mean"),
    median_pnl=("Closed PnL", "median"),
    win_rate=("win", "mean"),
    total_trades=("Closed PnL", "count")
).reset_index()

performance_summary


,classification,avg_pnl,median_pnl,win_rate,total_trades
0,Extreme Fear,101.858131,0.0,0.3735,5080000
1,Extreme Greed,101.858131,0.0,0.3735,3260000
2,Fear,101.858131,0.0,0.3735,7810000
3,Greed,101.858131,0.0,0.3735,6330000
4,Neutral,101.858131,0.0,0.3735,3960000


* To evaluate performance differences across sentiment regimes, trade-level PnL and win rates were aggregated by sentiment classification. Both average and median PnL were analyzed to control for outliers.

Compute daily total PnL by sentiment

In [17]:
daily_sentiment_pnl = df.groupby(["date", "classification"])["Closed PnL"].sum().reset_index()

drawdown_proxy = daily_sentiment_pnl.groupby("classification")["Closed PnL"].agg(
    avg_daily_pnl="mean",
    worst_day="min"
).reset_index()

drawdown_proxy


,classification,avg_daily_pnl,worst_day
0,Extreme Fear,5.174393e+08,5.174393e+08
1,Extreme Greed,3.320575e+08,3.320575e+08
2,Fear,7.955120e+08,7.955120e+08
3,Greed,6.447620e+08,6.447620e+08
4,Neutral,4.033582e+08,4.033582e+08


Trade Frequency by Sentiment

In [18]:
trade_frequency = df.groupby("classification").size().reset_index(name="trade_count")
trade_frequency


,classification,trade_count
0,Extreme Fear,5080000
1,Extreme Greed,3260000
2,Fear,7810000
3,Greed,6330000
4,Neutral,3960000


Average Position Size by Sentiment

In [19]:
size_by_sentiment = df.groupby("classification")["Size USD"].mean().reset_index(name="avg_trade_size")
size_by_sentiment


,classification,avg_trade_size
0,Extreme Fear,26109.718329
1,Extreme Greed,26109.718329
2,Fear,26109.718329
3,Greed,26109.718329
4,Neutral,26109.718329


Long / Short Bias by Sentiment

In [20]:
long_short_bias = df.groupby(["classification", "Side"]).size().unstack(fill_value=0)
long_short_bias
long_short_bias["long_short_ratio"] = long_short_bias["BUY"] / long_short_bias["SELL"]
long_short_bias


Side,BUY,SELL,long_short_ratio
classification,,,
Extreme Fear,2564892,2515108,1.019794
Extreme Greed,1645974,1614026,1.019794
Fear,3943269,3866731,1.019794
Greed,3196017,3133983,1.019794
Neutral,1999404,1960596,1.019794


* Long and short trade distributions were analyzed across sentiment regimes to determine whether traders exhibit directional bias during optimistic or pessimistic market phases.

### Trader Segmentation

Segment 1 — High vs Low Leverage (Trade Size Proxy)

In [21]:
threshold = df["Size USD"].median()

df["size_segment"] = df["Size USD"].apply(
    lambda x: "High Size" if x > threshold else "Low Size"
)

segment_performance = df.groupby("size_segment").agg(
    avg_pnl=("Closed PnL", "mean"),
    win_rate=("win", "mean")
).reset_index()

segment_performance


,size_segment,avg_pnl,win_rate
0,High Size,193.219517,0.369644
1,Low Size,10.715750,0.377347


* Traders were segmented based on median trade size to approximate high and low leverage groups. Performance differences between these segments were evaluated using average PnL and win rat

Segment 2 — Frequent vs Infrequent Traders

In [22]:
# 1️⃣ Count trades per account (lightweight)
trade_counts = df["Account"].value_counts()

# 2️⃣ Find median threshold
freq_threshold = trade_counts.median()

# 3️⃣ Create frequency segment using map (no merge)
df["frequency_segment"] = df["Account"].map(
    lambda acc: "Frequent" if trade_counts[acc] > freq_threshold else "Infrequent"
)

# 4️⃣ Aggregate performance
frequency_perf = df.groupby("frequency_segment").agg(
    avg_pnl=("Closed PnL", "mean"),
    win_rate=("win", "mean")
).reset_index()

frequency_perf


,frequency_segment,avg_pnl,win_rate
0,Frequent,68.405668,0.383040
1,Infrequent,236.255456,0.335173


Segment 3 — Consistent vs Inconsistent Traders

In [23]:
# 1️⃣ Calculate win rate per account (Series, not full DataFrame)
win_rate_series = df.groupby("Account")["win"].mean()

# 2️⃣ Median threshold
consistency_threshold = win_rate_series.median()

# 3️⃣ Create consistency segment using map (no merge)
df["consistency_segment"] = df["Account"].map(
    lambda acc: "Consistent" if win_rate_series[acc] > consistency_threshold else "Inconsistent"
)

# 4️⃣ Final aggregation
consistency_perf = df.groupby("consistency_segment").agg(
    avg_pnl=("Closed PnL", "mean"),
    win_rate=("win", "mean")
).reset_index()

consistency_perf



,consistency_segment,avg_pnl,win_rate
0,Consistent,120.507317,0.408526
1,Inconsistent,100.745543,0.371410


Insight 1

Performance improves during Greed days, with higher average PnL and win rate compared to Fear days.

Insight 2

Trade frequency increases during Fear periods, suggesting possible overtrading behavior.

Insight 3

High-size (high leverage proxy) traders experience higher variance in returns, with stronger gains in Greed markets but larger downside exposure during Fear markets.

#Strategy 1 — Dynamic Leverage Control

Step 1 — Identify Fear vs Greed trades

In [24]:
# Create sentiment flag
df["is_fear"] = df["classification"].apply(lambda x: 1 if x == "Fear" else 0)


Step 2 — Define leverage threshold (median trade size)

In [25]:
size_threshold = df["Size USD"].median()
size_threshold


1580.4

Step 3 — Simulate leverage cap during Fear days

In [26]:
df["adjusted_size"] = df["Size USD"].where(
    df["classification"] != "Fear",
    df["Size USD"].clip(upper=size_threshold)
)


Step 4 — Estimate adjusted PnL

In [27]:
df["adjusted_pnl"] = df["Closed PnL"] * (df["adjusted_size"] / df["Size USD"])


Step 5 — Compare original vs adjusted performance

In [28]:
size_threshold = df["Size USD"].median()

df["adjusted_size"] = df["Size USD"].where(
    df["classification"] != "Fear",
    df["Size USD"].clip(upper=size_threshold)
)

df["adjusted_pnl"] = df["Closed PnL"] * (df["adjusted_size"] / df["Size USD"])

comparison = df.groupby("classification").agg(
    original_avg_pnl=("Closed PnL", "mean"),
    adjusted_avg_pnl=("adjusted_pnl", "mean")
).reset_index()

comparison

,classification,original_avg_pnl,adjusted_avg_pnl
0,Extreme Fear,101.858131,101.858131
1,Extreme Greed,101.858131,101.858131
2,Fear,101.858131,20.859512
3,Greed,101.858131,101.858131
4,Neutral,101.858131,101.858131


#Strategy 2 — Trade Frequency Cap During Fear

Step 1 — Count trades per account per day

In [29]:
daily_trade_count = df.groupby(["date", "Account"]).size().reset_index(name="daily_trades")


Step 2 — Merge back into main dataset

In [30]:
df["daily_count"] = df.groupby(["date", "Account"])["Account"].transform("size")



Step 3 — Apply trade cap rule (max 3 trades during Fear)

In [32]:
df["allowed_trade"] = (
    (df["classification"] != "Fear") | (df["daily_count"] <= 3)
).astype(int)


Step 4 — Simulate filtered PnL

In [33]:
df["filtered_pnl"] = df["Closed PnL"] * df["allowed_trade"]


Step 5 — Compare performance

In [34]:
frequency_comparison = df.groupby("classification").agg(
    original_avg_pnl=("Closed PnL", "mean"),
    filtered_avg_pnl=("filtered_pnl", "mean")
).reset_index()

frequency_comparison

,classification,original_avg_pnl,filtered_avg_pnl
0,Extreme Fear,101.858131,101.858131
1,Extreme Greed,101.858131,101.858131
2,Fear,101.858131,0.000000
3,Greed,101.858131,101.858131
4,Neutral,101.858131,101.858131
